# Embeddings Evaluation

## Load in the embeddings

In [311]:

import pandas as pd
import numpy as np
import pickle
from itertools import islice
from sklearn import preprocessing  # Two samples, with 3 dimensions.


In [371]:
#embeddings_file="embeddings_full100_100_final.txt"
#embeddings_file="../../ull_ed/practical-2/embeddings_final_50_100_hansards_bsg.txt"
#embeddings_file="../../ull_ed/practical-2/embeddings_vocab_1613_ep_50_100_hid_100_test_bsg.txt"
pretrained = True
pretrained = False
if pretrained:
    embeddings_file="/Users/efiathieniti/Downloads/bow2.words"
    null_word = 'null'
else:
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1613_ep_20_100_hid_100_test_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1501_ep_40_emb_50_hid_100_True_test_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1601_ep_40_emb_100_hid_100_1600_test_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1551_ep_20_emb_100_hid_100_8_test_test_1550_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1401_ep_2_emb_100_hid_100_1400_test_bsg.txt"

    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1601_ep_8_emb_100_hid_100_4_test_test_1600_cat_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1601_ep_40_emb_100_hid_100_1600_test_bsg_good_with_common.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_1601_ep_8_emb_100_hid_100_128_test_test_1600_cat_bsg.txt"
    embeddings_file="../ull_ed/practical-2/embeddings_final_100_100_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_5533_ep_2_emb_100_hid_100_128_hansards_test_7000_cat_bsg.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_skipgram.txt"
    embeddings_file="../ull_ed/practical-2/output/embeddings_vocab_5001_5000_epochs_10_skipgram.txt"
    
    null_word = '<null>'


from gensim.models import KeyedVectors
#word_vectors = KeyedVectors.load_word2vec_format('../../../ULL/data/bow2.words.bz2', binary=False)  # C text format
word_vectors = KeyedVectors.load_word2vec_format(embeddings_file, binary=False, encoding='utf-8')  # C binary format


## Quantitative Comparison

### Evaluation against Simlex

In [ ]:
#word_vectors.evaluate_word_pairs("../gensim/gensim/test/test_data/wordsim353.tsv")



# Lexical substitution

In [ ]:
from nltk.tokenize import word_tokenize
import csv
path = '../../../Downloads/lst/'
lst_test_f = 'lst_test.preprocessed'
ls_candidates = 'lst.gold.candidates'
with open(path+lst_test_f,) as csvfile:
    reader = csv.reader(csvfile, delimiter='\t')
    sentences = []
    for row in reader:
        sentences.append(row)
         


In [ ]:
from nltk.tokenize import word_tokenize
import csv

# lstf file
# Columns: target word, id_sentence,    position of target word in sentence,    sentence

# Create the sentences to replace
# with open(path + lstf) as csvfile:
#     reader = csv.reader(csvfile)
#     sentences = []
#     for row in reader:
#         #sentences.append(row.split("\t"))
#         print(row)

sentences_df = pd.read_csv(path + lstf, sep="\t",  header=None, quoting=csv.QUOTE_NONE,
                       names = ["target","id_sentence","position","sentence"])


# Create the candidates dictionary
word_to_candidates = {}
with open(path+ls_candidates, 'r') as f:
    candidates = f.read().strip().split('\n')
    candidates = [tuple(line.split('::')) for line in candidates]
    for word, candids in candidates:
        cands = candids.split(';')
        w_cands = []
        for w in cands:
            if not ' ' in w:
                w_cands.append(w)
        word_to_candidates[word] = w_cands


In [ ]:

from nltk.tokenize import RegexpTokenizer

tokenized = []
contexts = []
for sent in sentences_df.sentence:
    tokenized.append(tokenizer.tokenize(sent))
    
sentences_df["tokenized"] = tokenized



In [ ]:
# Create the contexts based on window size
window_size = 5
contexts=[]
counter=0

for index, row in sentences_df.iterrows():
    pos = int(row["position"])
    target = row["target"]
    sent = row["tokenized"]
    words = [w if (w in word_vectors and not w.isdigit()) else null_word for w in sent]
    context = words[max(0, pos - window_size):min(len(words), pos + window_size)]
    contexts.append(context)
sentences_df['contexts'] = contexts



In [ ]:
 def write_rankings( rankings):
        out = ''
        for index, target, ranking in rankings:
            out += '#RANKED\t {} {}\t'.format(target, index)
            for candidate, score in ranking:
                out += '{} {}\t'.format(candidate, score)
            out = out[:-1] + '\n'
        with open('data/lst.out', 'w+') as f:
            f.write(out)

In [ ]:
# # ["target","id_sentence","position","sentence", "contexts"]
from operator import itemgetter

rankings = []
for index, row in sentences_df.iterrows():
    target = row["target"].split(".")[0]
    scores = []
    if target in word_vectors:
        for candidate in word_to_candidates[row["target"]]:
            if candidate not in word_vectors: cand_vec
            cand_vec = word_vectors[candidate] if candidate in word_vectors else word_vectors[null_word]
            word_vec = word_vectors[target] # obtain without .n
            # score against central word
            #np.random.shuffle(cand_vec)
            #np.random.shuffle(word_vec)
            score = analyze.cosine_similarity(cand_vec, word_vec)
            for context in row.contexts:
                context_vec= word_vectors[context]
                #np.random.shuffle(context_vec)

                score+=analyze.cosine_similarity(cand_vec, context_vec)
                score /= (len(row.contexts) + 1)

            # score for each candidate
            scores.append((candidate,score))
        scores.sort(key=itemgetter(1), reverse=True)
        rankings.append((row["id_sentence"],row["target"], scores))
    else:
        rankings.append((row["id_sentence"],row["target"],[]))


In [ ]:
write_rankings(rankings)

In [ ]:
rankings[1]